The Goal is to predict weapon type used based on the year, target type, country, and attack type

In [ ]:
# Import Libraries
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
import statsmodels.api as sm
import sklearn.model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [ ]:
# Read files
df = pd.read_excel('files/GTD/globalterrorismdb_0522dist.xlsx')

In [6]:
df_interest = df[['iyear', 'country_txt', 'attacktype1', 'targtype1', 'weaptype1_txt', 'nkill', 'nwound', 'success']]
df_cleaned = df_interest[df_interest['weaptype1_txt'] != 'Unknown']
df_cleaned = df_cleaned.dropna().reset_index()
display(df_cleaned)

,index,iyear,country_txt,attacktype1,targtype1,weaptype1_txt,nkill,nwound,success
0,5,1970,United States,2,3,Firearms,0.0,0.0,1
1,6,1970,Uruguay,1,3,Firearms,0.0,0.0,0
2,7,1970,United States,3,21,Explosives,0.0,0.0,1
3,8,1970,United States,7,4,Incendiary,0.0,0.0,1
4,9,1970,United States,7,2,Incendiary,0.0,0.0,1
...,...,...,...,...,...,...,...,...,...
173340,209699,2020,Yemen,3,12,Explosives,0.0,0.0,1
173341,209700,2020,Nepal,7,2,Incendiary,0.0,0.0,1
173342,209703,2020,Germany,7,4,Incendiary,0.0,0.0,1
173343,209704,2020,Afghanistan,2,14,Firearms,1.0,0.0,1


In [8]:
# liss attempt, linear regression 
import pandas as pd
import statsmodels.api as sm

# Create dummy variables 
country_dummies = pd.get_dummies(df['country_txt'], prefix='country', drop_first=True)
attack_type_dummies = pd.get_dummies(df['attacktype1_txt'], prefix='attack type', drop_first=True)
target_dummies = pd.get_dummies(df['targtype1_txt'],prefix='target type', drop_first=True)

# Concatenate the dummy variables with the DataFrame
new_df = pd.concat([df, country_dummies, attack_type_dummies, target_dummies], axis=1)

# Specify the columns to be included in X. Ensure all are numeric
X = new_df[['iyear', 'crit1'] + 
         attack_type_dummies.columns.tolist() + country_dummies.columns.tolist() + target_dummies.columns.tolist()].astype(float) 
y = new_df[['success']]

# Add a constant to the features to account for the intercept in the model
X_with_const = sm.add_constant(X)

# Fit the model to estimate the time for change regression 
model = sm.OLS(y, X_with_const).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                success   R-squared:                       0.169
Model:                            OLS   Adj. R-squared:                  0.168
Method:                 Least Squares   F-statistic:                     182.1
Date:                Mon, 26 Feb 2024   Prob (F-statistic):               0.00
Time:                        15:51:15   Log-Likelihood:                -39641.
No. Observations:              209706   AIC:                         7.975e+04
Df Residuals:                  209471   BIC:                         8.216e+04
Df Model:                         234                                         
Covariance Type:            nonrobust                                         
                                                      coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------

In [ ]:
df_encoded = pd.get_dummies(df_cleaned, columns=['country_txt', 'attacktype1', 'targtype1'])

X = df_encoded.drop('weaptype1_txt', axis=1)
y = df_encoded['weaptype1_txt']
X_intercept = sm.add_constant(X)

X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X_intercept, y, test_size=0.2, random_state=42)

model = LogisticRegression(max_iter=5).fit(X_train, y_train)

y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

In [ ]:
'''
enc = OneHotEncoder()
# Transforming categorical values to matrix
y = enc.fit_transform(y.values.reshape(-1,1))

x_cat = x['country_txt']
x_cat = enc.fit_transform(x_cat.values.reshape(-1,1))
x_cat = x_cat.toarray()

x_num = x.drop(columns={'country_txt'})
x_num = x_num.values

X = np.concatenate([x_cat, x_num], axis = 1)
'''